# Duplicados

Los duplicados en los datos (generalmente de toda una fila) no son necesariamente malos, ya que puede que los datos necesiten precisamente tener esos duplicados. Sin embargo, en general, siempre será bueno saber si existen y, en el caso de no quererlos, saber cómo eliminarlos.

Como es habitual, cargamos un `DataFrame` de referencia, que esta vez es algo especial porque sí contiene duplicados, aunque lo usaremos poco:


In [2]:
import numpy as np
import pandas as pd

In [3]:
df_aviones = pd.read_csv("C:/Users/david/Repo_Prueba/DS_Online_Febr25_Exercises/02_Programacion_y_Herramientas_Avanzadas/Sprint_04/Unidad_02/Practica_Obligatoria/data/dataset_viajes.csv",index_col="Id_vuelo")


## Detección y eliminación de filas duplicadas

La forma de detectar filas duplicadas es emplear el método `duplicated`. Este método se puede aplicar a todo el DataFrame o a una selección, y nos indicará si hay duplicados en esa selección.

La sintaxis para comprobar por filas es:

```python
df_aviones.duplicated()  # con un parámetro importante `keep`, que por defecto está a "first"

Este método devuelve una serie de booleanos donde True indica que la fila se considera duplicada y False que no se considera duplicada.

El criterio de Pandas para etiquetar una fila como duplicada depende del mencionado argumento `keep`:

1. **Etiqueta todas las filas que están duplicadas como duplicadas.** (`keep = False`)
2. **Etiqueta como duplicadas todas las apariciones menos la primera.** (`keep = "first"`)
3. **Etiqueta como duplicadas todas las apariciones menos la última.** (`keep = "last"`)

¿Y para qué tanto lío? Porque cuando queramos eliminar los duplicados, se eliminarán solo los etiquetados como tal, y así de esta forma podemos quedarnos con las primeras apariciones o las últimas.

Por ejemplo, supongamos un dataframe como:

In [4]:
df_dup = pd.DataFrame(
    {
        "col1": ["a", "c", "a", "a", "h", "c"],
        "col2": ["b", "d", "b", "b", "j", "d"],
        "col3": ["c", "a", "c", "c", "k", "a"]
    },
    index=["fila1", "fila2", "fila3", "fila4", "fila5", "fila6"]
)

df_dup

,col1,col2,col3
fila1,a,b,c
fila2,c,d,a
fila3,a,b,c
fila4,a,b,c
fila5,h,j,k
fila6,c,d,a


Donde las filas 1,3 y 4 estan duplicadas por un lado, y las filas 2 y 6 por otro. Veamos el efecto duplicated:

In [5]:
df_dup.duplicated () ### es igual a df_dup.duplicated(keep="first")

fila1    False
fila2    False
fila3     True
fila4     True
fila5    False
fila6     True
dtype: bool

Y si lo aplicamos como condición

In [6]:
df_dup.loc[df_dup.duplicated()]

,col1,col2,col3
fila3,a,b,c
fila4,a,b,c
fila6,c,d,a


Ahora con keep = last

In [7]:
df_dup.duplicated(keep="last")

fila1     True
fila2     True
fila3     True
fila4    False
fila5    False
fila6    False
dtype: bool

De nuevo veamos que nos selecciona como duplicados

In [8]:
df_dup.loc[df_dup.duplicated(keep="last")]

,col1,col2,col3
fila1,a,b,c
fila2,c,d,a
fila3,a,b,c


Y finalmente con keep = False

In [9]:
df_dup.loc[df_dup.duplicated(keep=False)]

,col1,col2,col3
fila1,a,b,c
fila2,c,d,a
fila3,a,b,c
fila4,a,b,c
fila6,c,d,a


Y todo esto es importante, porque el método para eliminar duplicados solo lo hará de aquellas filas que marquemos como duplicadas. Este método también tiene su parámetro `keep`, que implica la misma filosofía.

Veámoslo con `df_dup` y luego apliquemos a nuestro `DataFrame` de aviones.

(Primero nos hacemos una copia de backup)

In [10]:
df_dup_reserva = df_dup.copy()

In [11]:
df_dup.drop_duplicates() ## keep first

,col1,col2,col3
fila1,a,b,c
fila2,c,d,a
fila5,h,j,k


In [12]:
df_dup.drop_duplicates(keep="last") ## keep last

,col1,col2,col3
fila4,a,b,c
fila5,h,j,k
fila6,c,d,a


In [13]:
df_dup.drop_duplicates(keep=False) ## keep false

,col1,col2,col3
fila5,h,j,k


Si queremos que el metodo modifique el DataFrame que lo llama, debemos usar el argumento inplace con valor True

In [14]:
df_dup.drop_duplicates(keep=False,inplace=True) ## keep false

In [15]:
df_dup

,col1,col2,col3
fila5,h,j,k


Si lo aplicamos al DataFrame

In [16]:
##Veamos algunos duplicados

df_aviones.loc[df_aviones.duplicated(keep=False)]
df_aviones.loc["Mol_PaBa_10747"]

,Aircompany,Origen,Destino,Distancia,avion,consumo_kg,duracion
Id_vuelo,,,,,,,
Mol_PaBa_10747,MoldaviAir,París,Bali,11980.0,Boeing 747,134092.14,NaN
Mol_PaBa_10747,MoldaviAir,París,Bali,11980.0,Boeing 747,130260.936,818.0


In [17]:
##Podmos ver duplicados, filtrando por otros campos

In [18]:
df_aviones.loc[(df_aviones.Aircompany=="FlyQ")&(df_aviones.duplicated(keep=False))]

,Aircompany,Origen,Destino,Distancia,avion,consumo_kg,duracion
Id_vuelo,,,,,,,
Fly_BaBa_11380,FlyQ,Barcelona,Bali,13058.0,Airbus A380,166407.809152,1070.0
Fly_NuBa_11380,FlyQ,Nueva York,Bali,16589.0,Airbus A380,205422.781408,1305.0
Fly_NuBa_11380,FlyQ,Nueva York,Bali,16589.0,Airbus A380,205422.781408,1305.0
Fly_BaBa_11380,FlyQ,Barcelona,Bali,13058.0,Airbus A380,166407.809152,1070.0


In [19]:
##Finalmente veamos el agregado de duplicados en funcion del parametro keep

In [20]:
for val_keep in[False,"first","last"]:
    num_vuelos=len(df_aviones.loc[df_aviones.duplicated(keep=val_keep)])
    print(f"Para keep = {val_keep}")
    print(f"Numero de registros: = {num_vuelos}")
    

Para keep = False
Numero de registros: = 54
Para keep = first
Numero de registros: = 27
Para keep = last
Numero de registros: = 27


<!-- ## Detección de columnas -->

Para terminar, veamos cómo se pueden detectar duplicados en una o varias columnas, siempre teniendo en cuenta que este caso es aún más frecuente que el anterior. En general, salvo para **columnas** que deban tener valores únicos (como por ejemplo el DNI de una persona), no será muy útil comprobarlo.

```python


In [21]:
df_dup = df_dup_reserva.copy()  # recuperamos el anterior
df_dup

,col1,col2,col3
fila1,a,b,c
fila2,c,d,a
fila3,a,b,c
fila4,a,b,c
fila5,h,j,k
fila6,c,d,a


In [22]:
df_dup["col2"].duplicated()

fila1    False
fila2    False
fila3     True
fila4     True
fila5    False
fila6     True
Name: col2, dtype: bool

In [23]:
df_dup["col2"].duplicated(keep=False)

fila1     True
fila2     True
fila3     True
fila4     True
fila5    False
fila6     True
Name: col2, dtype: bool

In [24]:
df_dup.loc[df_dup["col2"].duplicated(keep=False)]

,col1,col2,col3
fila1,a,b,c
fila2,c,d,a
fila3,a,b,c
fila4,a,b,c
fila6,c,d,a


In [25]:
df_dup[["col2","col1"]].duplicated(keep="last")

fila1     True
fila2     True
fila3     True
fila4    False
fila5    False
fila6    False
dtype: bool

Si ahora queremos eliminar esas filas, usariamos drop_duplicates pero ojo nos cargariamos toda la fila y a lo mejor no es lo que nos interesa

In [28]:
df_dup.loc[df_dup[["col1","col2"]].duplicated(keep="last")]

,col1,col2,col3
fila1,a,b,c
fila2,c,d,a
fila3,a,b,c


## Nulos

A veces, en general muchas veces por no decir casi siempre, encontraremos en nuestros datos que no todas las columnas tienen valores (no ya que no sean correctos) sino que no tienen valores. Es decir, faltan datos.

Esta ausencia puede venir dada por un `""`, un valor vacío, o por un código especial. Además, es posible que cuando lo pasemos a Pandas nos genere en vez de ese `""`, vacío o código especial el ya antes mencionado **NaN**. Los **NaN** son una forma particular de expresar un valor vacío o un valor faltante.

### Detectando nulos y datos faltantes

Uno de los problemas de "echar un vistazo" (métodos `head` y `tail`) a un DataFrame es que así no es fácil detectar si hay datos faltantes o nulos o `NaN`. Por ejemplo:

[ ]:

In [29]:
df_aviones

,Aircompany,Origen,Destino,Distancia,avion,consumo_kg,duracion
Id_vuelo,,,,,,,
Air_PaGi_10737,Airnar,París,Ginebra,411.0,Boeing 737,NaN,51.0
Fly_BaRo_10737,FlyQ,Bali,Roma,12738.0,Boeing 737,33479.13254400001,1167.0
Tab_GiLo_11380,TabarAir,Ginebra,Los Angeles,9103.0,Airbus A380,NaN,626.0
Mol_PaCi_10737,MoldaviAir,París,Cincinnati,6370.0,Boeing 737,17027.01,503.0
Tab_CiRo_10747,TabarAir,Cincinnati,Roma,7480.0,Boeing 747,86115.744,518.0
...,...,...,...,...,...,...,...
Pam_LoNu_10747,PamPangea,Londres,Nueva York,5566.0,Boeing 747,"62300,238",391.0
Mol_MeLo_10747,MoldaviAir,Melbourne,Londres,16900.0,Boeing 747,194854.5664,1326.0
Mol_BaPa_10747,MoldaviAir,Bali,París,11980.0,Boeing 747,128983.868,818.0


In [30]:
df_aviones.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1000 entries, Air_PaGi_10737 to Air_PaCi_10737
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Aircompany  1000 non-null   object 
 1   Origen      1000 non-null   object 
 2   Destino     1000 non-null   object 
 3   Distancia   872 non-null    float64
 4   avion       1000 non-null   object 
 5   consumo_kg  862 non-null    object 
 6   duracion    853 non-null    float64
dtypes: float64(2), object(5)
memory usage: 94.8+ KB


Para verlo con mas precision, primero aplicaremos el metodo value_counts con un nuevo argumento: dropna

In [33]:
df_aviones["Destino"].value_counts(dropna=False)

Destino
Cincinnati     125
Bali           122
Londres        111
París          111
Ginebra        102
Nueva York     102
Roma            83
Los Angeles     62
Cádiz           58
Melbourne       52
Barcelona       21
GinEbra          2
GiNeBra          2
GINEbRa          2
GInEBrA          2
GInebra          2
BaRCelONa        2
GINebra          2
GInebRA          1
GINebRa          1
MelBOUrnE        1
BARcelOnA        1
GINEbRA          1
MelbOUrnE        1
MeLbourne        1
GIneBra          1
GIneBrA          1
GinebrA          1
GiNebrA          1
GiNEbRa          1
MeLboURnE        1
MELboURnE        1
MELBoURNe        1
MElboUrNe        1
MelboURNe        1
BARCEloNA        1
GInEBRa          1
GInEbRa          1
GINEBra          1
MelBoUrne        1
MelBourne        1
GineBra          1
MeLbOurne        1
GINEBrA          1
MelBouRne        1
MELBoURne        1
MELBourNe        1
GINeBrA          1
GiNEbra          1
MElbOUrnE        1
GiNEBra          1
BarcELONa        1
MElb

In [41]:
df_aviones["duracion"].value_counts(dropna=False)

duracion
NaN       147
845.0      26
818.0      26
1326.0     24
433.0      21
         ... 
123.0       1
731.0       1
533.0       1
151.0       1
488.0       1
Name: count, Length: 117, dtype: int64

Confirmada su existencia doblemente. La manera de identificar las filas con valores faltantes es a través del método `isna()` aplicado a las columnas en las que sabemos que hay valores nulos.

In [43]:
df_aviones.loc[df_aviones["duracion"].isna()]

,Aircompany,Origen,Destino,Distancia,avion,consumo_kg,duracion
Id_vuelo,,,,,,,
Mol_CaMe_10737,MoldaviAir,Cádiz,Melbourne,20029.0,Boeing 737,53148.15324000001,NaN
Tab_LoCi_10737,TabarAir,Los Angeles,Cincinnati,NaN,Boeing 737,"7915,433400000001",NaN
Pam_MeBa_10737,PamPangea,Melbourne,Bali,NaN,Boeing 737,7158.148200000001,NaN
Fly_GiCi_10737,FlyQ,Ginebra,Cincinnati,6969.0,Boeing 737,NaN,NaN
Mol_BaMe_11320,MoldaviAir,Bali,Melbourne,2779.0,Airbus A320,7114.795799999999,NaN
...,...,...,...,...,...,...,...
Mol_PaCi_11380,MoldaviAir,París,Cincinnati,6370.0,Airbus A380,79528.176,NaN
Pam_LoPa_11320,PamPangea,Londres,París,NaN,Airbus A320,949.784,NaN
Mol_MePa_11380,MoldaviAir,Melbourne,París,16925.0,Airbus A380,215687.8672,NaN


Si queremos ver las dos juntas

In [47]:
es_destino_Nan=df_aviones["duracion"].isna()
df_aviones.loc[es_destino_Nan]

,Aircompany,Origen,Destino,Distancia,avion,consumo_kg,duracion
Id_vuelo,,,,,,,
Mol_CaMe_10737,MoldaviAir,Cádiz,Melbourne,20029.0,Boeing 737,53148.15324000001,NaN
Tab_LoCi_10737,TabarAir,Los Angeles,Cincinnati,NaN,Boeing 737,"7915,433400000001",NaN
Pam_MeBa_10737,PamPangea,Melbourne,Bali,NaN,Boeing 737,7158.148200000001,NaN
Fly_GiCi_10737,FlyQ,Ginebra,Cincinnati,6969.0,Boeing 737,NaN,NaN
Mol_BaMe_11320,MoldaviAir,Bali,Melbourne,2779.0,Airbus A320,7114.795799999999,NaN
...,...,...,...,...,...,...,...
Mol_PaCi_11380,MoldaviAir,París,Cincinnati,6370.0,Airbus A380,79528.176,NaN
Pam_LoPa_11320,PamPangea,Londres,París,NaN,Airbus A320,949.784,NaN
Mol_MePa_11380,MoldaviAir,Melbourne,París,16925.0,Airbus A380,215687.8672,NaN


¿Y para todo el DataFrame

In [49]:
df_aviones.isna()

,Aircompany,Origen,Destino,Distancia,avion,consumo_kg,duracion
Id_vuelo,,,,,,,
Air_PaGi_10737,False,False,False,False,False,True,False
Fly_BaRo_10737,False,False,False,False,False,False,False
Tab_GiLo_11380,False,False,False,False,False,True,False
Mol_PaCi_10737,False,False,False,False,False,False,False
Tab_CiRo_10747,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...
Pam_LoNu_10747,False,False,False,False,False,False,False
Mol_MeLo_10747,False,False,False,False,False,False,False
Mol_BaPa_10747,False,False,False,False,False,False,False


## Tratamiento de Nulos/NaN (breve intro)

Aquí vamos a ver de una forma somera tres formas de tratar nulos:

1. Eliminar las filas o columnas con nulos.
2. Cambiar los valores de los nulos: Media.
3. Cambiar los valores de los nulos: Moda.

### Eliminar Nulos

Si no tenemos datos para algunas columnas y no sabemos como sustituirlos, entonces, lo mejor es no considerarlos, bien no considerar las columnas con nulos o bien eliminar toda la fila

### Eliminando columnas

In [50]:
df_aviones.drop(columns = ["duracion"])

,Aircompany,Origen,Destino,Distancia,avion,consumo_kg
Id_vuelo,,,,,,
Air_PaGi_10737,Airnar,París,Ginebra,411.0,Boeing 737,NaN
Fly_BaRo_10737,FlyQ,Bali,Roma,12738.0,Boeing 737,33479.13254400001
Tab_GiLo_11380,TabarAir,Ginebra,Los Angeles,9103.0,Airbus A380,NaN
Mol_PaCi_10737,MoldaviAir,París,Cincinnati,6370.0,Boeing 737,17027.01
Tab_CiRo_10747,TabarAir,Cincinnati,Roma,7480.0,Boeing 747,86115.744
...,...,...,...,...,...,...
Pam_LoNu_10747,PamPangea,Londres,Nueva York,5566.0,Boeing 747,"62300,238"
Mol_MeLo_10747,MoldaviAir,Melbourne,Londres,16900.0,Boeing 747,194854.5664
Mol_BaPa_10747,MoldaviAir,Bali,París,11980.0,Boeing 747,128983.868


In [51]:
df_aviones

,Aircompany,Origen,Destino,Distancia,avion,consumo_kg,duracion
Id_vuelo,,,,,,,
Air_PaGi_10737,Airnar,París,Ginebra,411.0,Boeing 737,NaN,51.0
Fly_BaRo_10737,FlyQ,Bali,Roma,12738.0,Boeing 737,33479.13254400001,1167.0
Tab_GiLo_11380,TabarAir,Ginebra,Los Angeles,9103.0,Airbus A380,NaN,626.0
Mol_PaCi_10737,MoldaviAir,París,Cincinnati,6370.0,Boeing 737,17027.01,503.0
Tab_CiRo_10747,TabarAir,Cincinnati,Roma,7480.0,Boeing 747,86115.744,518.0
...,...,...,...,...,...,...,...
Pam_LoNu_10747,PamPangea,Londres,Nueva York,5566.0,Boeing 747,"62300,238",391.0
Mol_MeLo_10747,MoldaviAir,Melbourne,Londres,16900.0,Boeing 747,194854.5664,1326.0
Mol_BaPa_10747,MoldaviAir,Bali,París,11980.0,Boeing 747,128983.868,818.0


Como ocurre con otros métodos "destructivos", el método `drop` no modifica el DataFrame que lo invoca, es necesario asignar el resultado o emplear el argumento `inplace=True`.

In [57]:
df_aviones_2 = df_aviones.copy()

df_aviones_2.drop(columns=["duracion"], inplace=True)
df_aviones_2

,Aircompany,Origen,Destino,Distancia,avion,consumo_kg
Id_vuelo,,,,,,
Air_PaGi_10737,Airnar,París,Ginebra,411.0,Boeing 737,NaN
Fly_BaRo_10737,FlyQ,Bali,Roma,12738.0,Boeing 737,33479.13254400001
Tab_GiLo_11380,TabarAir,Ginebra,Los Angeles,9103.0,Airbus A380,NaN
Mol_PaCi_10737,MoldaviAir,París,Cincinnati,6370.0,Boeing 737,17027.01
Tab_CiRo_10747,TabarAir,Cincinnati,Roma,7480.0,Boeing 747,86115.744
...,...,...,...,...,...,...
Pam_LoNu_10747,PamPangea,Londres,Nueva York,5566.0,Boeing 747,"62300,238"
Mol_MeLo_10747,MoldaviAir,Melbourne,Londres,16900.0,Boeing 747,194854.5664
Mol_BaPa_10747,MoldaviAir,Bali,París,11980.0,Boeing 747,128983.868


## Eliminando Filas

Si lo que queremos no es cargarnos toda la columna, porque consideramos que es un aspecto importante a considerar, deberemos eliminar las filas que no podamos cambiar sus valores NaN. En este caso, el método es **dropna** aplicado a filas:

[ ]:



In [63]:
df_aviones_2=df_aviones.copy()
df_aviones_2.dropna(axis="index",inplace=True)
df_aviones_2

,Aircompany,Origen,Destino,Distancia,avion,consumo_kg,duracion
Id_vuelo,,,,,,,
Fly_BaRo_10737,FlyQ,Bali,Roma,12738.0,Boeing 737,33479.13254400001,1167.0
Mol_PaCi_10737,MoldaviAir,París,Cincinnati,6370.0,Boeing 737,17027.01,503.0
Tab_CiRo_10747,TabarAir,Cincinnati,Roma,7480.0,Boeing 747,86115.744,518.0
Mol_PaLo_11320,MoldaviAir,París,Londres,344.0,Airbus A320,915.2464,44.0
Pam_PaMe_11380,PamPangea,París,Melbourne,16925.0,Airbus A380,217722.6584,1328.0
...,...,...,...,...,...,...,...
Pam_LoNu_10747,PamPangea,Londres,Nueva York,5566.0,Boeing 747,"62300,238",391.0
Mol_MeLo_10747,MoldaviAir,Melbourne,Londres,16900.0,Boeing 747,194854.5664,1326.0
Mol_BaPa_10747,MoldaviAir,Bali,París,11980.0,Boeing 747,128983.868,818.0


In [64]:
df_aviones_2.info

<bound method DataFrame.info of                 Aircompany      Origen     Destino  Distancia        avion  \
Id_vuelo                                                                     
Fly_BaRo_10737        FlyQ        Bali        Roma    12738.0   Boeing 737   
Mol_PaCi_10737  MoldaviAir       París  Cincinnati     6370.0   Boeing 737   
Tab_CiRo_10747    TabarAir  Cincinnati        Roma     7480.0   Boeing 747   
Mol_PaLo_11320  MoldaviAir       París     Londres      344.0  Airbus A320   
Pam_PaMe_11380   PamPangea       París   Melbourne    16925.0  Airbus A380   
...                    ...         ...         ...        ...          ...   
Pam_LoNu_10747   PamPangea     Londres  Nueva York     5566.0   Boeing 747   
Mol_MeLo_10747  MoldaviAir   Melbourne     Londres    16900.0   Boeing 747   
Mol_BaPa_10747  MoldaviAir        Bali       París    11980.0   Boeing 747   
Air_CaCi_10747      Airnar       Cádiz  Cincinnati     6624.0   Boeing 747   
Air_PaCi_10737      Airnar      

Te preguntarás, ¿y esto no sirve también para columnas? Sí, pero quería enseñarte el método `drop` así que he aprovechado… Pero podemos eliminar las columnas con nulos (todas, ojo, el `drop` es más selectivo) así.

In [67]:
df_aviones_2=df_aviones.copy()
df_aviones_2.dropna(axis="columns",inplace=True)#axis = 1

In [68]:
df_aviones_2

,Aircompany,Origen,Destino,Distancia,avion,consumo_kg,duracion
Id_vuelo,,,,,,,
Fly_BaRo_10737,FlyQ,Bali,Roma,12738.0,Boeing 737,33479.13254400001,1167.0
Mol_PaCi_10737,MoldaviAir,París,Cincinnati,6370.0,Boeing 737,17027.01,503.0
Tab_CiRo_10747,TabarAir,Cincinnati,Roma,7480.0,Boeing 747,86115.744,518.0
Mol_PaLo_11320,MoldaviAir,París,Londres,344.0,Airbus A320,915.2464,44.0
Pam_PaMe_11380,PamPangea,París,Melbourne,16925.0,Airbus A380,217722.6584,1328.0
...,...,...,...,...,...,...,...
Pam_LoNu_10747,PamPangea,Londres,Nueva York,5566.0,Boeing 747,"62300,238",391.0
Mol_MeLo_10747,MoldaviAir,Melbourne,Londres,16900.0,Boeing 747,194854.5664,1326.0
Mol_BaPa_10747,MoldaviAir,Bali,París,11980.0,Boeing 747,128983.868,818.0


## Sustitución por media y moda

Una opción frecuente es sustituir los valores numéricos por la media del resto de valores de la misma columna y los valores string por la moda del resto de valores de la columna. La moda es otra forma de decir "el valor más frecuente".

In [69]:
df_aviones_2 = df_aviones.copy()

In [70]:
df_aviones_2["Destino"].mode()

0    Cincinnati
Name: Destino, dtype: object

In [74]:
df_aviones.loc[df_aviones["Destino"].isna(),"Destino"]=df_aviones["Destino"].mode()

In [75]:
df_aviones.loc[df_aviones["duracion"].isna(),"duracion"]=df_aviones["duracion"].mean()

In [77]:
df_aviones.info()

<class 'pandas.core.frame.DataFrame'>
Index: 645 entries, Fly_BaRo_10737 to Air_PaCi_10737
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Aircompany  645 non-null    object 
 1   Origen      645 non-null    object 
 2   Destino     645 non-null    object 
 3   Distancia   645 non-null    float64
 4   avion       645 non-null    object 
 5   consumo_kg  645 non-null    object 
 6   duracion    645 non-null    float64
dtypes: float64(2), object(5)
memory usage: 56.5+ KB


In [81]:
df_aviones_2.loc[df_aviones_2["duracion"].isna()]

,Aircompany,Origen,Destino,Distancia,avion,consumo_kg,duracion
Id_vuelo,,,,,,,


In [82]:
df_aviones

,Aircompany,Origen,Destino,Distancia,avion,consumo_kg,duracion
Id_vuelo,,,,,,,
Fly_BaRo_10737,FlyQ,Bali,Roma,12738.0,Boeing 737,33479.13254400001,1167.0
Mol_PaCi_10737,MoldaviAir,París,Cincinnati,6370.0,Boeing 737,17027.01,503.0
Tab_CiRo_10747,TabarAir,Cincinnati,Roma,7480.0,Boeing 747,86115.744,518.0
Mol_PaLo_11320,MoldaviAir,París,Londres,344.0,Airbus A320,915.2464,44.0
Pam_PaMe_11380,PamPangea,París,Melbourne,16925.0,Airbus A380,217722.6584,1328.0
...,...,...,...,...,...,...,...
Pam_LoNu_10747,PamPangea,Londres,Nueva York,5566.0,Boeing 747,"62300,238",391.0
Mol_MeLo_10747,MoldaviAir,Melbourne,Londres,16900.0,Boeing 747,194854.5664,1326.0
Mol_BaPa_10747,MoldaviAir,Bali,París,11980.0,Boeing 747,128983.868,818.0
